In [2]:
import numpy as np
import sys, math, random
from collections import Counter


with open("/Users/mac/Desktop/code/tasksv11/en/qa1_single-supporting-fact_train.txt", mode="r") as f:
    raw = f.readlines()

tokens = []
for line in raw[0:1000]:
    
    line = line.replace('\t', '').replace('\n', '').replace('?', '').replace('.', '').replace('1','').\
    replace('2','').replace('3','').replace('4','').replace('5','').replace('6','').replace('7','').\
    replace('8','').replace('9','').replace('0', '')
    
    tokens.append(line.lower().split(" ")[1:])

vocab = set()
for sent in tokens:
    for word in sent:
        vocab.add(word)
vocab = list(vocab)
#print(len(vocab)) -19

word2index = {}
for index, word in enumerate(vocab):
    word2index[word] = index

def words2indices(sentence):
    idx = []
    for word in sentence:
        idx.append(word2index[word])
    return idx

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x/e_x.sum(axis=0)
    
np.random.seed(1)
embed_size = 10

embed = (np.random.rand(len(vocab), embed_size) * 0.5) * 0.1
reccurent = np.eye(embed_size)

start = np.zeros(embed_size)

decoder = (np.random.rand(embed_size, len(vocab)) * 0.5) * 0.1     #decoder.shape (10,19)

one_hot = np.eye(len(vocab))

def predict(sent):
    
    layers = list()
    layer = {}
    layer['hidden'] = start
    layers.append(layer)
    
    loss = 0
    
    for target_i in range(len(sent)):
        layer = {}
        
        #sent[target_i] -> red
        #layers[-1]['hidden'] -> вектор start
        layer['pred'] = softmax(layers[-1]['hidden'].dot(decoder))  # 1-10 x 10-19
        
        #layer['pred'].shape -> (19,)  прогноз вектора предложения
        
        loss += -np.log(layer['pred'][sent[target_i]])
        #print(layer['pred'][sent[target_i]])
        layer['hidden'] = layers[-1]['hidden'].dot(reccurent) + embed[sent[target_i]]
        
        layers.append(layer)
        
    return layers, loss


In [3]:
for iter in range(40000):
    alpha = 0.001
    sent = words2indices(tokens[iter%len(tokens)][1:])
    #sent = words2indices(tokens[5][1:])
    layers, loss = predict(sent)
    #print(len(vocab))
    #print(tokens[iter%len(tokens)][1:],tokens[iter%len(tokens)][:])
    
    for layer_idx in reversed(range(len(layers))):
        layer = layers[layer_idx]
        target = sent[layer_idx-1]
        #print('layer_idx', layer_idx)
        #print('layer:', layer)
        #print('target', target)
        #print('onehot', one_hot[target])
        
        #print(one_hot[2]) -> [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 
        
        if (layer_idx > 0):
            layer['output_delta'] = layer['pred'] - one_hot[target]
            new_hidden_delta = layer['output_delta'].dot(decoder.transpose())
        
            
            if (layer_idx == len(layers) - 1):
                layer['hidden_delta'] = new_hidden_delta
            else:
                layer['hidden_delta'] = new_hidden_delta + layers[layer_idx+1]['hidden_delta'].dot(reccurent.transpose())
        
        else:
            layer['hidden_delta'] = layers[layer_idx+1]['hidden_delta'].dot(reccurent.transpose())
                      
    start -= layers[0]['hidden_delta'] * alpha / float(len(sent))
    for layer_idx, layer in enumerate(layers[1:]):
        
        decoder -= np.outer(layers[layer_idx]['hidden'], layer['output_delta']) * alpha / float(len(sent))
        
        reccurent -= np.outer(layers[layer_idx]['hidden'], layer['hidden_delta']) * alpha / float(len(sent))
        
    if (iter%1000 == 0):
        print("Perplexity:" + str(np.exp(loss/len(sent))))

Perplexity:18.984618385977953
Perplexity:18.96275005587188
Perplexity:18.93877844404012
Perplexity:18.91042845970859
Perplexity:18.87457507827111
Perplexity:18.82642452292392
Perplexity:18.757899657589327
Perplexity:18.654040755636426
Perplexity:18.483848071440892
Perplexity:18.17277888208176
Perplexity:17.500489853797884
Perplexity:15.688562123214313
Perplexity:13.14588513029836
Perplexity:12.140259499740603
Perplexity:11.62391098031961
Perplexity:11.578062588141536
Perplexity:11.594517626270905
Perplexity:11.57212820079712
Perplexity:11.487371972762437
Perplexity:11.293462213728668
Perplexity:10.910145025194483
Perplexity:10.207273551828141
Perplexity:9.098117424679439
Perplexity:7.841592694590467
Perplexity:6.849939283163692
Perplexity:6.258363207029411
Perplexity:5.923915649024729
Perplexity:5.7213340295686255
Perplexity:5.582331292706548
Perplexity:5.471672677975867
Perplexity:5.376629772812686
Perplexity:5.294176289883838
Perplexity:5.222267616448059
Perplexity:5.157470206818019


In [ ]:
import numpy as np

def softmax(x_):
    x = np.atleast_2d(x_)
    temp = np.exp(x)
    return temp/np.sum(temp, axis=1, keepdims=True)

# word_vects = {}
# word_vects['yankees'] = np.array([[0,0,0]])
# word_vects['bears'] = np.array([[0,0,0]])
# word_vects['braves'] = np.array([[0,0,0]])
# word_vects['red'] = np.array([[0,0,0]])
# word_vects['sox'] = np.array([[0,0,0]])
# word_vects['lose'] = np.array([[0,0,0]])
# word_vects['defeat'] = np.array([[0,0,0]])
# word_vects['beat'] = np.array([[0,0,0]])
# word_vects['tie'] = np.array([[0,0,0]])

# sent2output = np.random.rand(3, len(word_vects))
# identity = np.eye(3)

# layer_0 = word_vects['red']
# layer_1 = layer_0.dot(identity) + word_vects['sox']
# layer_2 = layer_1.dot(identity) + word_vects['defeat']

# pred = softmax(layer_2.dot(sent2output))

# y = np.array([1,0,0,0,0,0,0,0,0])

# pred_delta = pred - y
# layer_2_delta = pred_delta.dot(sent2output.T)
# defeat_delta = layer_2_delta * 1
# layer_1_delta = layer_2_delta.dot(identity.T)
# sox_delta = layer_1_delta * 1
# layer_0_delta = layer_1_delta.dot(identity.T)
# alpha = 0.01
# word_vects['red'] -= layer_0_delta * alpha
# word_vects['sox'] -= sox_delta * alpha
# word_vects['defeat'] -= defeat_delta * alpha

# identity -= np.outer(layer_0, layer_1_delta) * alpha
# identity -= np.outer(layer_1, layer_2_delta) * alpha
# sent2output -= np.outer(layer_2, pred_delta) * alpha

